In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [13]:
np.random.seed(0)
df = pd.DataFrame()

umbrales = np.random.randint(50, 101, 10)

for i in range(10):
    print(f'Umbral #{i}:', umbrales[i])
    df[f'columna_{i}'] = np.random.randint(0, umbrales[i]+1, 1000)

df.head()

Umbral #0: 94
Umbral #1: 97
Umbral #2: 50
Umbral #3: 53
Umbral #4: 53
Umbral #5: 89
Umbral #6: 59
Umbral #7: 69
Umbral #8: 71
Umbral #9: 100


,columna_0,columna_1,columna_2,columna_3,columna_4,columna_5,columna_6,columna_7,columna_8,columna_9
0,36,6,16,35,39,16,4,33,52,98
1,87,89,3,1,34,57,40,47,9,82
2,70,88,26,29,39,5,25,14,24,20
3,88,38,6,20,48,8,50,31,46,26
4,88,51,41,37,44,33,37,63,45,28


In [14]:
t_stat, p_value = stats.ttest_ind(df['columna_0'], df['columna_1'])

t_stat, p_value

(-1.65488563397709, 0.09810475652975202)

In [15]:
data = pd.DataFrame({'media': df.mean(), 
                     'std': df.std(), 
                     'min': df.min(), 
                     'max': df.max(), 
                     'len':df.shape[0]
                    })
data = data.T

data

,columna_0,columna_1,columna_2,columna_3,columna_4,columna_5,columna_6,columna_7,columna_8,columna_9
media,46.997000,49.086000,25.294000,26.543000,26.608000,44.162000,29.185000,35.330000,34.610000,47.51200
std,28.137923,28.314596,14.773416,16.272337,15.734609,26.026278,17.214736,19.960148,20.239554,28.72112
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,94.000000,97.000000,50.000000,53.000000,53.000000,89.000000,59.000000,69.000000,71.000000,100.00000
len,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000


In [16]:
data.loc['media'][0] - data.loc['media'][1], data.loc['std'][0] - data.loc['std'][1]

(-2.0889999999999986, -0.17667212682867728)

In [17]:
media1 = data.loc['media'][0]
desvio1 = data.loc['std'][0]
tamano_muestra1 = data.loc['len'][0]

media2 = data.loc['media'][1]
desvio2 = data.loc['std'][1]
tamano_muestra2 = data.loc['len'][1]

diferencia_medias = media1 - media2
error_estandar_diferencia = ((desvio1**2 / tamano_muestra1) + (desvio2**2 / tamano_muestra2))**0.5

diferencia_medias, error_estandar_diferencia

(-2.0889999999999986, 1.2623228802703585)

In [18]:
t_stat = diferencia_medias / error_estandar_diferencia
t_stat

-1.65488563397709

In [19]:
grados_libertad = tamano_muestra1 + tamano_muestra2 - 2
grados_libertad

1998.0

In [20]:
p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df=grados_libertad))
p_value, round(p_value, 5)

(0.09810475652975215, 0.0981)

In [21]:
def ts_test(media1, desvio1, tamano_muestra1, media2, desvio2, tamano_muestra2):
    diferencia_medias = media1 - media2
    error_estandar_diferencia = ((desvio1**2 / tamano_muestra1) + (desvio2**2 / tamano_muestra2))**0.5
    t_stat = diferencia_medias / error_estandar_diferencia
    grados_libertad = tamano_muestra1 + tamano_muestra2 - 2
    p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df=grados_libertad))

    return t_stat, p_value

In [22]:
for i in range(data.shape[1]):
    for j in range(i+1, data.shape[1]):
        media1 = data.loc['media'][i]
        desvio1 = data.loc['std'][i]
        tamano_muestra1 = data.loc['len'][i]

        media2 = data.loc['media'][j]
        desvio2 = data.loc['std'][j]
        tamano_muestra2 = data.loc['len'][j]
        
        t_stat, p_value = ts_test(media1, desvio1, tamano_muestra1, media2, desvio2, tamano_muestra2)
        if p_value > 0.01:
            print(f'columna_{i} vs columna_{j}: t_stat: {round(t_stat, 5)} | p_value: {round(p_value, 5)} ----> ATENCION!')
        else:
            print(f'columna_{i} vs columna_{j}: t_stat: {round(t_stat, 5)} | p_value: {round(p_value, 5)}')

columna_0 vs columna_1: t_stat: -1.65489 | p_value: 0.0981 ----> ATENCION!
columna_0 vs columna_2: t_stat: 21.59533 | p_value: 0.0
columna_0 vs columna_3: t_stat: 19.89926 | p_value: 0.0
columna_0 vs columna_4: t_stat: 19.99959 | p_value: 0.0
columna_0 vs columna_5: t_stat: 2.33898 | p_value: 0.01943 ----> ATENCION!
columna_0 vs columna_6: t_stat: 17.07577 | p_value: 0.0
columna_0 vs columna_7: t_stat: 10.69445 | p_value: 0.0
columna_0 vs columna_8: t_stat: 11.30123 | p_value: 0.0
columna_0 vs columna_9: t_stat: -0.40504 | p_value: 0.68549 ----> ATENCION!
columna_1 vs columna_2: t_stat: 23.55794 | p_value: 0.0
columna_1 vs columna_3: t_stat: 21.82882 | p_value: 0.0
columna_1 vs columna_4: t_stat: 21.94366 | p_value: 0.0
columna_1 vs columna_5: t_stat: 4.04876 | p_value: 5e-05
columna_1 vs columna_6: t_stat: 18.99158 | p_value: 0.0
columna_1 vs columna_7: t_stat: 12.5568 | p_value: 0.0
columna_1 vs columna_8: t_stat: 13.15263 | p_value: 0.0
columna_1 vs columna_9: t_stat: 1.23413 | p_va

In [23]:
data

,columna_0,columna_1,columna_2,columna_3,columna_4,columna_5,columna_6,columna_7,columna_8,columna_9
media,46.997000,49.086000,25.294000,26.543000,26.608000,44.162000,29.185000,35.330000,34.610000,47.51200
std,28.137923,28.314596,14.773416,16.272337,15.734609,26.026278,17.214736,19.960148,20.239554,28.72112
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,94.000000,97.000000,50.000000,53.000000,53.000000,89.000000,59.000000,69.000000,71.000000,100.00000
len,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
